In [1]:
# Architecture
# https://www.tensorflow.org/extend/architecture

In [2]:
# Hands on useful core OPs of Tensorflow
# tf

In [4]:
# (1) Sessions

In [10]:
# sess = tf.Session() - returns graph_pb2.GraphDef contatining nodes definition
# sess.run()
# sess.close()

# Using the context manager which modifies default session to the lifetime of context
# with tf.Session() as sess:
#   sess.run()
#   variable.eval() - shows that the operation is evaluating in this session

# sess.as_default() - Returns a context manager that makes this object the default session
# with sess.as_default():
#   assert tf.get_default_session() is sess
# tf.get_default_session - Gets current default session

# sess.run(fetches, feed_dict={}) exexutes a graph registered with this session
# fetches - can be tensor, string, operation
# returns list of fetches values

# sess = tf.InteractiveSession() - installs itself as the default session. We don't have to define it again and again 
# The methods tf.Tensor.eval and tf.Operation.run will use that session to run ops.

In [11]:
# (2) Graphs (quick overview)
# A Graph contains a set of tf.Operation objects, which represent units of computation and 
# tf.Tensor objects, which represent the units of data that flow between operations.

# A default Graph is always registered, and accessible by calling tf.get_default_graph
# g.default_graph() - default graph can be overridden by this context manager which 
# overrides the current default graph for the lifetime of the context
# Use this method with the with keyword to specify that ops 
# created within the scope of a block should be added to this graph only and no other graph
# this allows us to exectute more than one graph simultaneously

# g = tf.Graph()
# with g.as_default():
#   # Define operations and tensors in `g`.
#   c = tf.constant(30.0)
#   assert c.graph is g

# graph supports collections - collections can store groups of related objects like losses, variables etc
# GraphKeys - https://www.tensorflow.org/api_docs/python/tf/GraphKeys
# add_to_collection(name, value) - adds new data item to its list of available collections

# .as_graph_def() - Returns a serialized GraphDef representation of this graph.

# .as_graph_element(obj) - Returns the object referred to by obj, as an Operation or Tensor.
# This function validates that obj represents an element of this graph, and gives error if not

# clear_collection(name) - clears the named value
# get_all_collection_keys() - gets the list of collections
# get_collection(name, scope=None) - gets the value for named key, always creates a copy of named key and returns
# get_collection_ref() - always returns original list

# colocate_with() - Returns a context manager that specifies an op to colocate with.
# a = tf.Variable([1.0])
# with g.colocate_with(a):
#   b = tf.constant(1.0)
#   c = tf.add(a, b)
# here whereever a is present it will always be associated with b and c

# container() - stateful operations like queues and variables can maintain there states and can be used across
# devices with same states. so we create container for similar types and track there states with them
# with g.container('variable_add'):
#   v1 = tf.Variable([1.0])
#   v2 = tf.Variable([2.0])
#   v3 = v1 + v2
#   v4 = v1 + 10
# with g.container('variable_mul'):
#   v5 = tf.Variable([11.0])
#   v6 = tf.Variable([32.0])
#   v7 = v5 * v6
#   v8 = v1 * 8

# control_dependencies(control_inputs) - all operations constructed within this context should run only after
# control_inputs has been exececuted first
# with g.control_dependencies([a, b, c]):
#   d = d + 10
#   e = e * d
# pass None to clear control dependency in case of nested dependencies.

# .Device() - default device to use
#. get_operation_by_name(name) - Returns the Operation with the given name.
# .get_tensor_by_name(name) - Returns the Tensor with the given name.

# GraphKeys - https://www.tensorflow.org/api_docs/python/tf/GraphKeys

In [13]:
# (3) Sharing Variables
# .name_scope(*args, **kwds) - creates a stack of hierarchical names of operations
# .variable_scope() - creates a stack of hierarchical names of variables
# http://stackoverflow.com/questions/35919020/whats-the-difference-of-name-scope-and-a-variable-scope-in-tensorflow
# this basically ensures that rather than variables are created everytime, old ones are used as it is to ensure proper 
# weight sharing between operations/graphs 
# http://stackoverflow.com/questions/34592172/about-names-of-variable-scope-in-tensorflow
# http://stackoverflow.com/questions/33725159/understanding-variable-scope-example-in-tensorflow
# retrieving a variable using its scope
# with tf.variable_scope("param"):
#    w = tf.get_variable("weights", [1])
#    print("w:", w.name)
# name_scope() - on tf.Variable only
# variable_scope() - on tf.get_variable also along with tf.Variable
# https://www.tensorflow.org/programmers_guide/variable_scope
# tf.get_variable_scope().reuse_variables() - for reusing same variable instead of re-initalising it

In [14]:
# (4) Variables
# https://www.tensorflow.org/api_guides/python/state_ops#Exporting_and_Importing_Meta_Graphs

In [15]:
# (5) Some useful operations
# tf.argmax() - returns max along axis - used in predictions
# tf.one_hot() - encodes labels to 1 rest to 0 by taking index and depth of tensor
# tf.train.write_graph(sess.graph, '/tmp/my-model', 'train.pbtxt') -  writes the graph's pbtxt file
# tf.train.global_step()
# tf.train.get_global_step()
# tf.losses.softmax_cross_entropy(onehot_labels, logits, loss_collection=tf.GraphKeys.LOSSES)
# tf.losses.get_losses() - Gets the list of losses from the loss_collection.
# tf.losses.get_total_loss(add_regularization_losses=True, name='total_loss') - Returns a tensor whose value represents the total loss.
# tf.losses.sigmoid_cross_entropy(multi_class_labels, logits, weights=1.0) - same as loss but calculates sigmoid loss

In [16]:
# (6) Saver
# https://www.tensorflow.org/api_docs/python/tf/train/Saver
# stores and retrieves lastes checkpoint
# saver = tf.Saver()
# saver.save(sess, 'my-model', global_step=0) # global step is optional # returns path that can be passed to restore
# saver.restore() # restores the last checkpoint 
# tf.train.latest_checkpoint(checkpoint_dir, latest_filename=None) - grab latest checkpoint
# tf.train.get_checkpoint_state(checkpoint_dir, latest_filename=None) - gets latest state from checkpoint file

In [17]:
# (7) Summary
#  tf.summary.FileWriter - writes a summary to event file. Updates asynchronously.
# This event file will contain Event protocol buffers constructed when you call one of the following functions: 
# add_summary(), add_session_log(), add_event(), or add_graph().
# writer = tf.summary.FileWriter(<some-directory>, sess.graph)
# writer.add_session_log(session_log, global_step=None) - adds session log
# tf.summary.tensor_summary - adds tensor summary as serialized proto
# tf.summary.scalar() - adds single scalar unit  like loss, accuracy etc.
# tf.summary.histogram() - when we want to add list of data to see it overtime like activation fuctions etc
# similarly image, audio
# all summaries are added to GraphKeys.SUMMARIES
# tf.summary.merge - merges all the summary op into one
# tf.summary.merge_all - Merges all summaries collected in the default graph in GraphKeys.SUMMARIES

In [3]:
# (8) Most used operations - 
# https://www.tensorflow.org/api_guides/python/train - gradient optimizers
# https://www.tensorflow.org/api_guides/python/nn - layers used func such as convd, relu etc
# https://www.tensorflow.org/api_docs/python/tf/losses - losses to be used

In [4]:
# (9) optimizers
# operations that are responsible for learning
# The Optimizer base class provides methods to compute gradients for a loss and apply gradients to trainable variables
# every obtimizer is associated with minimize() property that is responsible fo calculating and updating gradients of 
# variable. gate_gradients (optional) - IT has GATE_NONE, GATE_OP, GATE_GRAPH for parallelism. Default is GraphKey.TRAINABLE_VARIABLES

# loss = tf.reduce_mean(tf.square(layer2 - marks)) or tf.losses.mean_squared_error(labels=layer2, predictions=marks)
# optimizer = tf.train.AdamOptimizer(0.5)
# train = optimizer.minimize(loss)

# If you want custom then do this - 
# 1) create optimizer
# 2) pass the list to for optimizing, default is list of all trainable variables
# 3) compute gradients - if you want you own computation else leave it on algo
# 4) apply gradients - to apply the gradients across variables

# opt = GradientDescentOptimizer(learning_rate=0.1)
# grads_and_vars = opt.compute_gradients(loss, <list of variables>)
# capped_grads_and_vars = [(MyCapper(gv[0]), gv[1]) for gv in grads_and_vars]
# opt.apply_gradients(capped_grads_and_vars)

In [5]:
# (10) weight decay-
# decays the learning rate  while training - 
# tf.train.exponential_decay
# tf.train.inverse_time_decay
# tf.train.natural_exp_decay
# tf.train.piecewise_constant
# tf.train.polynomial_decay

# learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100000, 0.96, staircase=True)
# tf.train.GradientDescentOptimizer(learning_rate).minimize(...my loss..., global_step=global_step)
